In [ ]:
import rasterio
import scipy.interpolate as spip


def get_get_alt(dem_name):
    dem = rasterio.open(dem_name)
    band = dem.read(1)

    def get_alt(lon, lat):
        r, c = dem.index(lon, lat)
        x, y = dem.xy(r, c)
        dx, dy = dem.res[0], dem.res[1]
        X = [x - dx, x, x + dx]
        Y = [y + dy, y, y - dy]
        Z = band[r - 1 : r + 2, c - 1 : c + 2]
        return spip.interp2d(X, Y, Z)(lon, lat)[0]

    return get_alt

In [ ]:
from shapely.geometry import LineString

get_alt = get_get_alt("./data/_hakone.vrt")


def get_geom_3d(row):
    coords = row.geometry.coords[:]
    if row["type"] == "トンネル":
        a = round(get_alt(*coords[0]), 3)
        return LineString([(c[0], c[1], a) for c in coords])
    else:
        return LineString([(c[0], c[1], round(get_alt(*c), 3)) for c in coords])

In [ ]:
import geopandas as gpd


def arrange_df(df):
    return df.query("ftCode in ('2701','2703','2704') & rnkWidth!='3m未満'").drop(
        [
            "lfSpanFr",
            "lfSpanTo",
            "tmpFlg",
            "orgGILvl",
            "admCode",
            "devDate",
            "state",
            "lvOrder",
            "admOfcRd",
            "Width",
            "sectID",
            "tollSect",
            "medSect",
            "motorway",
            "repLtdLvl",
            "rtCode",
            "name",
            "comName"
        ],
        axis=1,
    )

df = arrange_df(gpd.read_file("./data/_hakone.gpkg", layer="rdcl"))
df.to_file('./data/_rdcl.geojson', index=False, driver='GeoJSON')
# CLON, CLAT = 139.103528, 35.233333
# WIDTH = 20 / 3600
# df = df.cx[CLON - WIDTH : CLON + WIDTH, CLAT - WIDTH : CLAT + WIDTH]

In [ ]:
import json
from functools import partial
import numpy as np
from pyproj import Transformer

T = Transformer.from_crs(4612, 2451, always_xy=True)


def get_edge(reverse, row):
    def get_distances(coords):
        coords = list(T.itransform(coords))
        lengths = np.sqrt(np.sum(np.diff(np.array(coords), axis=0) ** 2, axis=1))
        return [sum(lengths[:i]) for i in range(len(lengths) + 1)]

    g = row.geometry
    coords = g.coords[::-1] if reverse else g.coords[:]
    distances = get_distances(coords)
    attr = dict(row)
    del attr["geometry"]
    return [
        tuple(coords[0][:2]),
        tuple(coords[-1][:2]),
        attr | {"length": distances[-1], "coords": coords[:], "distances": distances},
    ]

df.geometry = df.apply(get_geom_3d, axis=1)
edges = (
    df.apply(partial(get_edge, False), axis=1).to_list()
    + df.apply(partial(get_edge, True), axis=1).to_list()
)
json.dump(edges, open("./data/_roads_nw.json", "w"), ensure_ascii=False, indent=2)

In [ ]:
import networkx as nx

ax = df.plot()
g = nx.MultiDiGraph()
g.add_edges_from(edges)
nx.draw(g, {n: [n[0], n[1]] for n in list(g.nodes)}, node_size=5, ax=ax)

In [ ]:
import contextily as cx

ax = df.plot()
cx.add_basemap(ax, crs=df.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)

In [10]:
import json
import networkx as nx

edges = [[tuple(start_node), tuple(end_node), prop] for (start_node, end_node, prop) in json.load(open('./data/_roads_nw.json'))]
G = nx.Graph()
G.add_edges_from(edges)

In [11]:
G